## Automation

***

## Project Description

Exercise 1 - Automate loading of incremental data into the data warehouse

### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random

import datetime
from datetime import datetime, timedelta
import scipy.stats

import sqlite3 as sq3

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
#sets the default autosave frequency in seconds
%autosave 60 
sns.set_style('dark')
sns.set(font_scale=1.2)

plt.rc('axes', titlesize=9)
plt.rc('axes', labelsize=14)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)

pd.set_option('display.max_columns',None)
#pd.set_option('display.max_rows',None)
pd.set_option('display.width', 1000)
pd.option_context('float_format','{:.2f}'.format)

random.seed(0)
np.random.seed(0)
np.set_printoptions(suppress=True)

Autosaving every 60 seconds


## Upload Data to Database

In [2]:
con = sq3.connect("automation.db")

In [3]:
con

In [4]:
sales = pd.read_csv("sales.csv")
sales.head()

,rowid,product_id,customer_id,price,quantity,timestamp
0,104,8972,58516,4277,5,2017-11-22 03:53:23
1,105,7905,59794,542,2,2017-11-22 03:53:24
2,106,6295,86874,2706,2,2017-11-22 03:53:25
3,107,6665,56517,4821,1,2017-11-22 03:53:26
4,108,8851,23457,4612,3,2017-11-22 03:53:27


In [5]:
sales.to_sql("sales_data", con, index=False)  #Load to database

In [6]:
con.execute("Select * FROM sqlite_master").fetchall()

[('table',
  'sales_data',
  'sales_data',
  2,
  'CREATE TABLE "sales_data" (\n"rowid" TEXT,\n  "product_id" TEXT,\n  "customer_id" TEXT,\n  "price" TEXT,\n  "quantity" TEXT,\n  "timestamp" TEXT\n)')]

In [7]:
pd.read_sql_query("Select * FROM sqlite_master", con) #Check all tables inside database

,type,name,tbl_name,rootpage,sql
0,table,sales_data,sales_data,2,"CREATE TABLE ""sales_data"" (\n""rowid"" TEXT,\n ..."


In [8]:
con.close()

## Load Data from SQL database

### SQLite

In [9]:
db = sq3.connect("automation.db") #Create connection

In [10]:
cursor = db.cursor()

In [11]:
# List all the tables in the database.

cursor = db.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")

available_table=(cursor.fetchall())

available_table

[('sales_data',)]

## Write SQL Queries

In [12]:
df = pd.read_sql_query("SELECT * FROM sales_data", db)
df

,rowid,product_id,customer_id,price,quantity,timestamp
0,104,8972,58516,4277,5,2017-11-22 03:53:23
1,105,7905,59794,542,2,2017-11-22 03:53:24
2,106,6295,86874,2706,2,2017-11-22 03:53:25
3,107,6665,56517,4821,1,2017-11-22 03:53:26
4,108,8851,23457,4612,3,2017-11-22 03:53:27
...,...,...,...,...,...,...
12193,12285,7895,69638,902,5,2017-11-22 07:16:24
12194,12286,7262,79625,150,4,2017-11-22 07:16:25
12195,12287,6268,78558,2320,3,2017-11-22 07:16:26
12196,12288,8191,53404,2834,4,2017-11-22 07:16:27


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12198 entries, 0 to 12197
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   rowid        12198 non-null  object
 1   product_id   12198 non-null  object
 2   customer_id  12198 non-null  object
 3   price        12198 non-null  object
 4   quantity     12198 non-null  object
 5   timestamp    12198 non-null  object
dtypes: object(6)
memory usage: 571.9+ KB


# Find out the last rowid from DB2 data warehouse
# The function get_last_rowid must return the last rowid of the table sales_data on the IBM DB2 database.

In [14]:
def get_last_rowid(df):
    return df.iloc[-1]


In [15]:
last_row_id = get_last_rowid(df)
print("Last row id on production datawarehouse = ", last_row_id)

Last row id on production datawarehouse =  rowid                         12289
product_id                     6512
customer_id                    9400
price                          3486
quantity                          4
timestamp       2017-11-22 07:16:28
Name: 12197, dtype: object


# List out all records in MySQL database with rowid greater than the one on the Data warehouse
# The function get_latest_records must return a list of all records that have a rowid greater than the last_row_id in the sales_data table in the sales database on the MySQL staging data warehouse.

In [16]:
def get_latest_records(rowid):
    return np.where(df.loc[:,"rowid"] > "12000", 1,0)


In [17]:
new_records = get_latest_records(df)

print("New rows on staging datawarehouse = ", len(new_records))

New rows on staging datawarehouse =  12198


# Insert the additional records from MySQL into DB2 data warehouse.
# The function insert_records must insert all the records passed to it into the sales_data table in IBM DB2 database.

In [21]:
def insert_records():
    return pd.read_sql_query("INSERT INTO sales_data(rowid,'product_id',customer_id,price,quantity,timestamp)\
                       VALUES('12290','0000','00000','0','0','2017-11-22 07:16:29')", con=db)
    

In [22]:
insert_records()
#print("New rows inserted into production datawarehouse = ", len())

TypeError: 'NoneType' object is not iterable

In [23]:
cursor.close()

***

#### Python code done by Dennis Lam